In [9]:
from bloqade.gemini import logical as gemini_logical
from bloqade import qubit, squin
from bloqade.lanes.logical_mvp import (
    compile_to_physical_stim_program,
    kernel,
    set_detector,
    set_observable,
    TesseractDecoder,
    GeminiLogical,
)
import numpy as np
from scipy.linalg import block_diag

In [10]:
@kernel
def main():
    # see arXiv: 2412.15165v1, Figure 3a
    reg = qubit.qalloc(5)
    squin.broadcast.t(reg)

    squin.broadcast.sqrt_x([reg[0], reg[1], reg[4]])
    squin.broadcast.cz([reg[0], reg[2]], [reg[1], reg[3]])
    squin.broadcast.sqrt_y([reg[0], reg[3]])
    squin.broadcast.cz([reg[0], reg[3]], [reg[2], reg[4]])
    squin.sqrt_x_adj(reg[0])
    squin.broadcast.cz([reg[0], reg[1]], [reg[4], reg[3]])
    squin.broadcast.sqrt_y_adj(reg)

    measurements = gemini_logical.terminal_measure(reg)

    for i in range(len(reg)):
        set_detector([measurements[i]])
        set_observable(measurements[i], i)

Here, `measurements` is of type `list[list[MeasurementResult]]`. Thus, the `terminal_measure` statement introduces physical bits in the logical kernel. These physical bits are then accessed directly in the `set_detector` and `set_observable` statements to define the detectors and observables.

Once a kernel is defined, it can be submitted for execution to the backend, using an API similar to the following:

In [11]:
service = GeminiLogical()
future = service.submit(main, shots=100)
results = future.get_results()

In [12]:
results.phyiscal_program.diagram(height=400)

The results struct contains the following fields:
- `measurement_bits`: The measurement bits.
- `detector_bits`: The detector bits.
- `logical_bits`: The logical bits.
- `detector_error_model`: The detector error model.



In [13]:
results.detector_error_model[:5]

stim.DetectorErrorModel('''
    error(0.00841086) D0 D1 D2
    error(0.00109209) D0 D1 D2 D3 D4 D5
    error(0.00230601) D0 D1 D2 D3 D4 D5 D6 D7 D8
    error(0.000285679) D0 D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 D11
    error(0.000285679) D0 D1 D2 D3 D4 D5 D9 D10 D11
''')

In [14]:
decoder = TesseractDecoder(results.detector_error_model)

In [15]:
corrected_logical_bits = decoder.decode(results.detector_bits, results.logical_bits)
corrected_logical_bits

array([[ True,  True, False,  True, False],
       [ True,  True, False,  True, False],
       [ True,  True, False, False, False],
       [False, False,  True,  True, False],
       [ True,  True, False,  True, False],
       [ True, False, False,  True,  True],
       [False,  True,  True,  True,  True],
       [ True,  True, False, False, False],
       [False, False, False, False, False],
       [False,  True, False,  True,  True],
       [ True,  True, False, False, False],
       [ True, False,  True,  True,  True],
       [ True, False, False,  True,  True],
       [ True, False,  True,  True,  True],
       [False,  True,  True,  True, False],
       [False, False, False, False, False],
       [ True,  True,  True, False,  True],
       [False,  True, False, False,  True],
       [False,  True,  True, False, False],
       [False, False,  True,  True, False],
       [False,  True, False, False,  True],
       [False,  True, False,  True,  True],
       [False, False,  True,  Tr

## Underlying pipeline

The noise model is generated by `compile_to_physical_stim_program` which consumes a logical kernel and produces a STIM circuit with noise annotations which are derived from the hardware-compiled kernel.

In [16]:
program = compile_to_physical_stim_program(main)

In [17]:
print(f"""
{program[:5]}
...
{program[-5:]}
""")


U3(0.0, 0.0, 0.03978874) 6
SQRT_Y_DAG 0 1 2 3 4 5
CZ 1 2 3 4 5 6
SQRT_Y 6
CZ 0 3 2 5 4 6
...
OBSERVABLE_INCLUDE(3) rec[-14] rec[-13] rec[-9]
DETECTOR(0, 0) rec[-7] rec[-6] rec[-5] rec[-4]
DETECTOR(0, 1) rec[-6] rec[-5] rec[-3] rec[-2]
DETECTOR(0, 2) rec[-5] rec[-4] rec[-3] rec[-1]
OBSERVABLE_INCLUDE(4) rec[-7] rec[-6] rec[-2]



In [18]:
program.diagram("timeline-svg", height=400)

We use STIM to generate a detector error model from the physical circuit.

In [19]:
dem = program.detector_error_model(approximate_disjoint_errors=True)
dem[:5]

stim.DetectorErrorModel('''
    error(0.00841086) D0 D1 D2
    error(0.00109209) D0 D1 D2 D3 D4 D5
    error(0.00230601) D0 D1 D2 D3 D4 D5 D6 D7 D8
    error(0.000285679) D0 D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 D11
    error(0.000285679) D0 D1 D2 D3 D4 D5 D9 D10 D11
''')

## Measurement-to-detector matrices

Alternatively, specify matrices `m2obs` and `m2dets` that convert measurements to detectors and observables, respectively.

`detector_bits = measurement_bits @ m2dets`

`observable_bits = measurement_bits @ m2obs`



In [20]:
d = np.array([[1,1,1,1,0,0,0], [0,1,1,0,1,1,0], [0,0,1,1,1,0,1]])
o = np.array([[1,1,0,0,0,1,0]])

m2obs = block_diag(*[o.T]*5)  # shape (35, 5) - 35 measurements, 5 observables
m2dets = block_diag(*[d.T]*5)  # shape (35, 15) - 35 measurements, 15 detectors

In [21]:
@kernel
def main2():
    reg = qubit.qalloc(5)
    squin.broadcast.t(reg)

    squin.broadcast.sqrt_x([reg[0], reg[1], reg[4]])
    squin.broadcast.cz([reg[0], reg[2]], [reg[1], reg[3]])
    squin.broadcast.sqrt_y([reg[0], reg[3]])
    squin.broadcast.cz([reg[0], reg[3]], [reg[2], reg[4]])
    squin.sqrt_x_adj(reg[0])
    squin.broadcast.cz([reg[0], reg[1]], [reg[4], reg[3]])
    squin.broadcast.sqrt_y_adj(reg)
    
    squin.broadcast.measure(reg)


In [ ]:
service = GeminiLogical()
future = service.submit(main2, m2obs=m2obs, m2dets=m2dets, shots=100)
results = future.get_results()

In [ ]:
print(f"""
{results.phyiscal_program[:5]}
...
{results.phyiscal_program[-5:]}
""")


U3(0.0, 0.0, 0.03978874) 6
SQRT_Y_DAG 0 1 2 3 4 5
CZ 1 2 3 4 5 6
SQRT_Y 6
CZ 0 3 2 5 4 6
...
DETECTOR rec[-35] rec[-34] rec[-33] rec[-32]
DETECTOR rec[-35] rec[-34] rec[-33] rec[-32]
DETECTOR rec[-35] rec[-34] rec[-33] rec[-32]
DETECTOR rec[-35] rec[-34] rec[-33] rec[-32]
DETECTOR rec[-35] rec[-34] rec[-33] rec[-32]



## Memory Experiment

Different kernels require different detectors. Here, an example of a memory experiment. If transversal logic was present between stabilizer readout, detector definitions had to be adjusted (as transversal gate map stabilizers to different stabilizers)

In [ ]:
@kernel
def memory_main():
    reg = qubit.qalloc(5)
    data = reg[0]
    aux_x = [reg[1], reg[3]]
    aux_z = [reg[2], reg[4]]

    num_rounds = 2

    squin.broadcast.h(aux_x)
    for i in range(num_rounds):  # stabilizer rounds
        squin.cnot(aux_x[i], data)
        squin.cnot(data, aux_z[i])
    squin.broadcast.h(aux_x)
    
    measurements = gemini_logical.terminal_measure(reg)
    m_data = measurements[0]
    m_aux_x = [measurements[1], measurements[3]]
    m_aux_z = [measurements[2], measurements[4]]

    # detector and observable annotations
    set_observable(m_data)

    # for stabilizer readout, detectors compare stabilizer of two subsequent rounds
    set_detector([m_aux_z[0]])
    for i in range(num_rounds - 1):
        set_detector([m_aux_z[i], m_aux_z[i + 1]])
        set_detector([m_aux_x[i], m_aux_x[i + 1]])
    set_detector([m_data, m_aux_z[-1]])